In [27]:
using CellBasedModels
using CairoMakie
Makie.inline!(true);

In [45]:
include("physiCell.jl");

In [42]:
model = ABM(3,
    baseModelInit = [
        volumeModel,
        cellCycleLiveCellsModel,
        cellDeathApoptosisModel,
        cellDivisionModel
    ]
);

In [44]:
com = Community(model, N=10, dt=0.1)

Community with 10 agents.


In [52]:
for params in [parametersVolumeModel, parametersLiveCellCycleModel, parametersDeathApoptosisModel]
    for p in keys(params)
        println(p)
#        com[p] = params[p]
    end
end

fCN
rC
v
rF
fF
vN
rN
vNStarget
rb
rNApop
rFApop
rCApop
